# CrewAI framework

<img src="images/crewai_main.png" width=600px />

**Почему CrewAI?**

🤼‍♀️ Role-Playing Agents: Агенты могут принимать на себя различные роли и образы, чтобы лучше понять и взаимодействовать со сложными системами.

🤖 Autonomous Decision Making: Агенты могут принимать решения автономно на основе заданного контекста и доступных инструментов.

🤝 Seamless Collaboration: Агенты могут работать вместе без каких-либо затруднений, обмениваясь информацией и ресурсами для достижения общих целей.

🧠 Complex Task Tackling: CrewAI предназначен для решения сложных задач, таких как многоступенчатые рабочие процессы, принятие решений и разрешение проблем.

## 1. Agents, Tools, Tasks

In [ ]:
# !pip -qqq install 'crewai[tools]'==0.51.0
# !pip install pandas
# !pip install langchain-together==0.1.5


In [1]:
import json
import os
import sqlite3
from datetime import datetime, timezone
from pathlib import Path
from textwrap import dedent
from typing import Any, Dict, List, Tuple, Union
import dotenv

import pandas as pd
from crewai import Agent, Crew, Process, Task
from crewai_tools import tool

from langchain_community.tools.sql_database.tool import (
    InfoSQLDatabaseTool,
    ListSQLDatabaseTool,
    QuerySQLCheckerTool,
    QuerySQLDataBaseTool,
)

from langchain_community.utilities.sql_database import SQLDatabase
from langchain_core.callbacks.base import BaseCallbackHandler
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from langchain_openai import ChatOpenAI

from langchain_community.llms.yandex import YandexGPT
from langchain_together import ChatTogether, Together

dotenv.load_dotenv(override=True)

True

In [2]:
df = pd.read_csv("datasets/ds-salaries.csv")
df.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


In [ ]:
connection = sqlite3.connect("db/salaries.db")
df.to_sql(name="salaries", con=connection)

In [3]:
# llm = ChatTogether(
#     temperature=0,
#     model='meta-llama/Meta-Llama-3.1-8B-Instruct-Turbo',
#     max_tokens=1024,
#     verbose=True
# )

# llm = ChatGroq(
#     temperature=0.2,
#     model="llama3-groq-8b-8192-tool-use-preview",
#     max_tokens=4096)

llm = ChatOpenAI(
    temperature=0.2,
    model="gpt-4o-mini",
    max_tokens=4096)


# groq_llm = LLM(
#     model="groq/llama3-8b-8192",
#     temperature=0.3,
#     max_tokens=4096,
#     api_key="gsk_",
#     base_url="https://api.groq.com/"
# )

In [4]:
LANGCHAIN_TRACING_V2 = True
LANGCHAIN_ENDPOINT="https://api.smith.langchain.com"
LANGCHAIN_API_KEY="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"
LANGCHAIN_PROJECT="pr-rundown-experiment-5"

llm.invoke('Кто тебя создал? Ответь коротко.')

AIMessage(content='Меня создала компания OpenAI.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 18, 'total_tokens': 26, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0705bf87c0', 'finish_reason': 'stop', 'logprobs': None}, id='run-9f62efad-a1ab-420e-9660-7a6d4c9d4bf7-0', usage_metadata={'input_tokens': 18, 'output_tokens': 8, 'total_tokens': 26})

In [5]:
human = "{text}. Answer in russian."
prompt = ChatPromptTemplate.from_messages([("human", human)])

chain = prompt | llm
response = chain.invoke(
    {
        "text": "У кого сейчас самая лучшая LLM? Ответь только названием компании. Объясни почему."
    }
)

print(response.content)

На данный момент самой лучшей LLM считается OpenAI. 

Причины: OpenAI активно развивает свои модели, такие как GPT-4, которые демонстрируют высокую производительность в различных задачах обработки естественного языка, включая понимание контекста, генерацию текста и решение сложных задач. Компания также уделяет внимание безопасности и этическим аспектам использования ИИ, что делает их технологии более надежными и приемлемыми для широкого круга пользователей.


### 1.1 Creating tools

In [6]:
db = SQLDatabase.from_uri("sqlite:///db/salaries.db")

@tool("list_tables")
def list_tables() -> str:
    """List the available tables in the database"""
    return ListSQLDatabaseTool(db=db).invoke("")

@tool("tables_schema")
def tables_schema(tables: str) -> str:
    """
    Input is a comma-separated list of tables, output is the schema and sample rows
    for those tables. Be sure that the tables actually exist by calling `list_tables` first!
    Example Input: table1, table2, table3
    """
    return InfoSQLDatabaseTool(db=db).invoke(tables)

@tool("execute_sql")
def execute_sql(sql_query: str) -> str:
    """Execute a SQL query against the database. Returns the result"""
    return QuerySQLDataBaseTool(db=db).invoke(sql_query)

@tool("check_sql")
def check_sql(sql_query: str) -> str:
    """
    Use this tool to double check if your query is correct before executing it. Always use this
    tool before executing a query with `execute_sql`.
    """
    return QuerySQLCheckerTool(db=db, llm=llm).invoke({"query": sql_query})

In [ ]:

list_tables.run()

In [ ]:
print(tables_schema.run("salaries"))

In [ ]:
execute_sql.run("SELECT * FROM salaries WHERE salary > 10000 LIMIT 5")


In [ ]:
check_sql.run("SELECT * WHERE salary > 10000 LIMIT 5 table = salaries")

## 2. Agents

In [7]:
sql_dev = Agent(
    role="Senior Database Developer",
    goal="Construct and execute SQL queries based on a request",
    backstory=dedent(
        """
        You are an experienced database engineer who is master at creating efficient and complex SQL queries.
        You have a deep understanding of how different databases work and how to optimize queries.
        Use the `list_tables` to find available tables.
        Use the `tables_schema` to understand the metadata for the tables.
        Use the `execute_sql` to check your queries for correctness.
        Use the `check_sql` to execute queries against the database.
    """
    ),
    llm=llm,
    tools=[list_tables, tables_schema, execute_sql, check_sql],
    allow_delegation=False,
    verbose=True
)

data_analyst = Agent(
    role="Senior Data Analyst",
    goal="You receive data from the database developer and analyze it",
    backstory=dedent(
        """
        You have deep experience with analyzing datasets using Python.
        Your work is always based on the provided data and is clear,
        easy-to-understand and to the point. You have attention
        to detail and always produce very detailed work (as long as you need).
    """
    ),
    llm=llm,
    allow_delegation=False,
)
   
report_writer = Agent(
    role="Senior Report Editor",
    goal="Write an executive summary type of report based on the work of the analyst",
    backstory=dedent(
        """
        Your writing still is well known for clear and effective communication.
        You always summarize long texts into bullet points that contain the most
        important details.
        """
    ),
    llm=llm,
    allow_delegation=False,
)

## 3. Tasks

In [8]:
extract_data = Task(
    description="Extract data that is required for the query {query}.",
    expected_output="Database result for the query",
    agent=sql_dev,
)

analyze_data = Task(
    description="Analyze the data from the database and write an analysis for {query}.",
    expected_output="Detailed analysis text",
    agent=data_analyst,
    context=[extract_data],
)

write_report = Task(
    description=dedent(
        """
        Write an executive summary of the report from the analysis. The report
        must be less than 100 words. Answer in Russian.
    """
    ),
    expected_output="Markdown report",
    agent=report_writer,
    context=[analyze_data],
)

## 4. Create Crew

In [9]:
crew = Crew(
    agents=[sql_dev, data_analyst, report_writer],
    tasks=[extract_data, analyze_data, write_report],
    process=Process.sequential,
    verbose=True,
    memory=False,)


In [10]:
inputs = {
    "query": "Effects on salary (in USD) based on company location, size and employee experience"
}

result = crew.kickoff(inputs=inputs)

2024-11-28 15:12:26,104 - 4544071168 - manager.py-manager:293 - WARNING: Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


 [2024-11-28 15:12:26][DEBUG]: == Working Agent: Senior Database Developer
 [2024-11-28 15:12:26][INFO]: == Starting Task: Extract data that is required for the query Effects on salary (in USD) based on company location, size and employee experience.
I need to start by identifying the available tables in the database to understand what data I can work with for the query regarding the effects on salary based on company location, size, and employee experience.

Action: list_tables
Action Input: {} 

salaries

Thought: There is only one table available, which is "salaries." I need to check the schema of this table to understand its structure and the relevant columns for the query regarding salary effects based on company location, size, and employee experience.

Action: tables_schema
Action Input: {"tables": "salaries"} 


CREATE TABLE salaries (
	"index" INTEGER, 
	work_year INTEGER, 
	experience_level TEXT, 
	employment_type TEXT, 
	job_title TEXT, 
	salary INTEGER, 
	salary_currency TE

In [11]:
print(result)

### Исполнительное резюме анализа влияния местоположения компании, размера и опыта сотрудников на зарплаты

- **Данные**: Анализ зарплат по странам, размерам компаний и уровням опыта сотрудников.
- **Размер компании**:
  - Большие: средняя зарплата $83,000.
  - Средние: средняя зарплата $54,000.
  - Малые: средняя зарплата $43,000.
- **Уровень опыта**:
  - Начальный: $45,000.
  - Средний: $55,000.
  - Старший: $70,000.
  - Исполнительный: $150,000.
- **Ключевые страны**: США, Германия, Австралия, Индия.
- **Выводы**: Зарплаты зависят от размера компании, уровня опыта и географического положения.
